<a href="https://colab.research.google.com/github/ImranNust/AppliedDataScience-CapstonProject/blob/main/00_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atrium Segmentation

In this lecture series, we will build a neural network which will automatically segment the left atrium in cardiac MRI images as usual.

- Segmenting and image is the process of assigning a specific class to all pixels or voxels in an image. In our case, this means that each voxel is either classified as 'not left atrium' or 'left atrium'.

- This segmentation of the atrium allows to exactly calculate its volume. Changes in atrial volume are associated with multiple cardiac disorders, such as atrial fibrillation or mitral valve stenosis, which basically is a narrowing of the mitral valve orifice blocking blood flow to the heart chambers. However, as you might already imagine, Manuel's, like, is an extremely tedious task because the radiologist has to manually recommend the left atrium in all slices where it is visible.

- Therefore, let's automates it.

---

## Data

We can use the cardiac MRI dataset provided by the medical segmentation they and the medical segmentation decathlon, or in short, MSD is a challenge consisting of 10 different medical segmentation tasks. Atrium segmentation is one of them.

- The corresponding dataset consists of 20 MRI scans of the heart with corresponding ground truth masks in the two dimensional setting.

- It's up to 4542 2D slices. 



- You can download and preprocess the data for the segmentation task for cardiac mri images:
  - The data is provided by the medical segmentation decathlon (http://medicaldecathlon.com/)
  - (Data License: https://creativecommons.org/licenses/by-sa/4.0/)

- You can directly download the cardiac MRIs and segmentation maps from:
https://drive.google.com/file/d/1wEB2I6S6tQBVEPxir8cA5kFB8gTQadYY/view?usp=sharing

---

## Preprocessing


In general, there are two ways we could tackle this problem.

1. The first possibility is to work in the three dimensional setting by using 3D convolutions. However, using three dimensional convolutions is extremely memory hungry and computational expensive. Therefore, let's work in a two dimensional setting by extracting the single slices.

2. Our original image shape is 252 squared. However, as we are only interested in the cardiac regions, we could crop away the noncardiac regions and the background by removing the first 32 rows or columns from all borders. Please note that you'll also have to crop the segmentation masks next.

    - We prefer a z-normalization on the subject level. This means that we compute mean and standard deviation for each subject separately before said, normalizing the corresponding skin.

    - Subsequently, we standardize the normalized subject into the $[0, 1]$ range by performing minimax scaling to evaluate the China realization ability of our model.
    
    $$ X_s = \frac{X_n - min(X_n)}{max(X_n) - min(X_n)} $$


- We only use 16 patients as training data and the remaining four for validation purposes.

- After pre-processing our data, it's time to talk about the dataset.

    

---

## Datasets

1. At first, our dataset needs to create a list of all two dimensional slices.

2. Next, given an index, it has to load displays and corresponding label mask.

3. After loading our data, We have to augment slice and mask identically.

4. Last but not least, we need to return the augmented slice and mask.

**Data Augmentation**

- we use scaling,  rotation and elastic transformations for augmentation.

- Elastic transformations augment an image by moving the pixels locally around using a displacement field.

Before talking about the training routine, let's discuss the model we will manually implement.

---
## Model


- One of the most famous segmentation models in this lecture called UNet.

- The UNet was proposed by Raymond Berger at All Atomic Height 2015.

- It consists of an encoder decoder architecture with skip connections.

- The encoder has the task of reducing the feature Maps by using normal convolutions and max pooling layers.

- In contrast, the decoder has to reconstruct the segmentation mask based on the original image and the features by using upsampling, layers and also normal convolutions.

- Last but not least, the use of Skip connections allows an information flow directly from the encoder to the decoder.

- This drastically reduces the problem of vanishing gradients.



---

## Training


- We will train our model by using the Adam optimizer with an initial learning rate of 0.0001.

- Together with the currently most used loss for segmentation loss called dice loss, the dice loss is

$$L(\hat{y}, y) = 1 - \frac{2|\hat{y} \cap y|}{|\hat{y}|+|\hat{y}|}$$

- In code, as we are working in the binary setting, we can simply compute the intersection in the counter by calculating the sum of the element twice product between prediction and label. The denominator is simply calculated by adding the sum of the predictions to the sum of the masks. Additionally, as both the prediction and mask might be zero, we add a really small number to the denominator
```
counter = (pred * mask).sum() # Numerator
denum = pred.sum() + mask.sum() + 1e-8 # Denominator
dice = 1 - (2*counter)/denum
```

- Threshold at 0.5
    - Predictions > 0.5  -> Atrium
    - Predictions <= 0.5 -> Not Atrium



- we train our model for 75 epochs.